In [1]:
import os
import warnings
import pickle
import numpy as np
import random

os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

In [2]:
with open(r"./Data/dataset_n306_X_500_12.pickle", "rb") as input_file:
    dataset = pickle.load(input_file)

(x_dataset, y_dataset, labels) = dataset
permutation = list(range(x_dataset.shape[0]))

# if you need to shuffle the dataset
random.shuffle(permutation)

x_dataset = np.array([x_dataset[i] for i in permutation])
y_dataset = np.array([y_dataset[i] for i in permutation])
labels = np.array([labels[i] for i in permutation])

leads = [6, 7, 8] # V1 V2 V3
n_leads = len(leads)
x_dataset = np.take(x_dataset, leads, axis=-1)

In [ ]:
import numpy as np

from search import *
from Models.Ensemble.EnsembleVoter import EnsembleVoter
from Models.model_buiders import build_ESN
from generators import grid_hp_generator

n_out_fold = 5
n_restart = 2

splitted_x_dataset = np.split(x_dataset, n_leads, axis=-1)

m_list = [[] for _ in range(n_out_fold)]
kf_out = KFold(n_splits=n_out_fold)

with open(fr"./ensemble_{n_leads}.txt", "w") as output_file:
    for fold in range(n_out_fold):
        train_val_index, test_index = list(kf_out.split(x_dataset))[fold]
        print(f"fold {fold+1}")
        y_trainval, y_test = y_dataset[train_val_index], y_dataset[test_index]
        x_trainval = []
        x_test = []
        for i in range(n_leads):
            x_trainval.append(splitted_x_dataset[i][train_val_index])
            x_test.append(splitted_x_dataset[i][test_index])

        for rest in range(n_restart):
            print(f"restart {rest+1}")
            ESNs = []
            for i, x_set in enumerate(x_trainval):
                print(f"lead {i}")
                ESN, param, train, val = search_direct(build_ESN,
                                                               grid_hp_generator(n_iter=108, units=[50, 1000],
                                                                                 input_scaling=[1.5],
                                                                                 bias_scaling=[0.1],
                                                                                 spectral_radius=[0.999], leaky=[0.1]),
                                                               x_set, y_trainval, k_fold_val_reg, n_fold=4)
                print("ESN n.", i+1, file=output_file)
                print("units", param["units"], file=output_file)
                print("input_scaling", param["input_scaling"], file=output_file)
                print("bias_scaling", param["bias_scaling"], file=output_file)
                print("spectral radius", param["spectral_radius"], file=output_file)
                print("leaking rate", param["leaky"], file=output_file)
                print("regression lambda", param["reg"], file=output_file)
                print("training accuracy", train, file=output_file)
                print("validation accuracy", val, file=output_file)
                t = ESN.evaluate_precise(x_test[i], y_test)
                print("test accuracy", t[0], file=output_file)
                print("sensitivity", t[1], file=output_file)
                print("specificity", t[2], file=output_file)
                ESNs.append(ESN)
            print("", file=output_file)
            voting_model = EnsembleVoter(ESNs)
            train_acc = voting_model.evaluate(x_trainval, y_trainval)
            test_acc = voting_model.evaluate_precise(x_test, y_test)
            print("train accuracy", train_acc, file=output_file)
            print("test accuracy", test_acc[0], file=output_file)
            print("sensitivity", test_acc[1], file=output_file)
            print("specificity", test_acc[2], file=output_file)
            print("", file=output_file)
            m_list[fold].append((voting_model, "", train_acc, "", test_acc))
            print("", file=output_file)



In [12]:
train_accuracys = []
test_accuracys = []
sens_accuracys = []
specs_accuracys = []

for mf in m_list:
    trains = np.array([m[2] for m in mf])
    tests = np.array([m[4][0] for m in mf])
    sens = np.array([m[4][1] for m in mf])
    specs = np.array([m[4][2] for m in mf])
    train_accuracys.append(trains.mean())
    test_accuracys.append(tests.mean())
    sens_accuracys.append(sens.mean())
    specs_accuracys.append(specs.mean())

train_accuracys = np.array(train_accuracys)
test_accuracys = np.array(test_accuracys)
sens_accuracys = np.array(sens_accuracys)
specs_accuracys = np.array(specs_accuracys)

final_train_avg = train_accuracys.mean()
final_train_std = train_accuracys.std()
final_test_avg = test_accuracys.mean()
final_test_std = test_accuracys.std()
final_sens_avg = sens_accuracys.mean()
final_sens_std = sens_accuracys.std()
final_spec_avg = specs_accuracys.mean()
final_spec_std = specs_accuracys.std()

with open(fr"./result_ensemble_{n_leads}.txt", "w") as output_file:
    print("leads: ", leads, file=output_file)
    print("train accuracy: ", final_train_avg, file=output_file)
    print("train accuracy std: ", final_train_std, file=output_file)
    print("test accuracy: ", final_test_avg, file=output_file)
    print("test accuracy std: ", final_test_std, file=output_file)
    print("sensitivity accuracy: ", final_sens_avg, file=output_file)
    print("sensitivity accuracy std: ", final_sens_std, file=output_file)
    print("specificity accuracy: ", final_spec_avg, file=output_file)
    print("specificity accuracy std: ", final_spec_std, file=output_file)

    print(
        f"${round(final_train_avg, 3)} \pm {round(final_train_std, 4)}$ & ${round(final_test_avg, 3)} \pm {round(final_test_std, 4)}$ & ${round(final_sens_avg, 3)} \pm {round(final_sens_std, 4)}$ & ${round(final_spec_avg, 3)} \pm {round(final_spec_std, 4)}$",
        file=output_file)
    print("", file=output_file)